<a href="https://colab.research.google.com/github/emerenan/xlsx_validation/blob/main/script_vba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import datetime as dt

log_output = '''
{sheetname}                   Columns Validation
------------------------------------------------
# of Columns received: {total_received}
# of Columns required for billing: {number_missing_columns}
# of Columns transformed from file: 

Name of Missing columns in {sheetname} spreadsheet:
{columns}
'''

def read_files(path, sheetname, n_skiprows, n_skip_columns, columns_integer_convert):
    """
    Params:\n
    path: parth of file in the computer.\n
    n_skiprows: Number of rows to delete in the original file,.\n
    columns_to_convert: Columns to convert the data general type. \n
    n_skipcolumn: Columns to skip in the original file. \n
    endrow = pass 0 to read everything, 1 to count entire
    columns_order: List of columns names in specific order to pass in the engine.\n
    """
    # read file and skip header
    df = pd.read_excel(path, sheet_name = sheetname, skiprows = n_skiprows)
    # skip blank columns to create df
    df = df.iloc[:,n_skip_columns:]

    # define the entiry columns which doesn't have NaN 
    cont = 0
    for i in df.iloc[:,0]:
        if str(i) == 'nan':
            break
        else:
            cont +=1
    df = df.iloc[:cont, :]

    #df.columns = columns_order
    
    # convert intery columns to integer 
    df[columns_integer_convert] = df[columns_integer_convert].fillna(0)
    df[columns_integer_convert] = df[columns_integer_convert].astype('int64')

    
    for i in columns_integer_convert:
        lista = []
        for index in df[i]:
            if index == 0:
                lista.append('')
            else:
                lista.append(index)
        df[i] = lista
    
    #print(endrow)
    return df

def replace_values(df, column, conditional_value, value_to_change):
    """
    Está voltando para float
    """

    lista = []
    for index in df[column]:
        if index == conditional_value:
            lista.append(value_to_change)
        else:
            lista.append(index)
    df[column] = lista
    

def change_date_format(df, columns_date_convert, format):
    for i in columns_date_convert:
        if len(df[i].value_counts()) > 0:
            df[i] = df[i].dt.strftime(format)

    return df

def check_picklist(df, column, picklist):

    errors = 0
    diff_value = []
    for i in df[column]:
        if i not in picklist:
            errors += 1
            diff_value.append(i)
    
    output=f"""
    Check picklist of column: {column}
    ----------------------------------
    # of error found: {errors}
    different picklist value: {diff_value}
    """
    print(output)

def on_air_sites_check(df, columns, status_column, status, path_write):
    
    # dataframe with a conditional setted
    status_df = df[df[status_column]==status]

    # Empty Dataframe to receive blank lines
    new_df = pd.DataFrame()

    aux = []
    for i in columns:
        df = status_df[status_df[i].isnull()]
        if not df.empty:
            aux.append(df)
    if aux != []:
        new_df = pd.concat(aux)
        new_df.drop_duplicates(inplace=True)
        new_df.to_excel(path_write, index=False)


def check_columns(table, output_columns):
    """
    Check the total of number of missing columns and the missing columns in passed table.\n

    Params:\n
    table: contain the columns to be check.\n
    output_columns: columns structure at the final file. 

    Returns:\n
    Number of missing columns and a list that contains the name os missing columns.
    """
    """"
    countries = ['DE':{'towerDB':[],\
                       'lc': [],\
                       'ta':[]}\
                 ,'HU','IE','RO','PT','ES','CZ','GR']"""
    total_received = len(table.columns)
    number_missing_columns = 0
    missing_columns = []

    #Counting of missing columns       
    #if country in contries:      
    for columns in output_columns:
        if columns.lower() not in [labels.lower() for labels in table]:
            number_missing_columns +=1
            missing_columns.append(columns)
    
    return total_received, number_missing_columns, missing_columns



In [3]:
# Lendo o Ficheiro de input
#path='/content/Vantage Towers_PT TowerDB Apr21.xlsx'
test = '/content/teste_dates.xlsx'
sheetname = 'Final Delivery'
skiprows = 6
skipcolumns = 1
list_columns = ["Site Code","Site Name","EVO Site ID","Macro Region","Region","Province","Municipality","Nr. Inhabitants","Address","Altitude","Latitude","Longitude","Ground Register","Categorization by inhabitants","Categorization by Transmission Sys","Categorization by Transmission Sys (sub-cluster)","Categorization by Site Type","Other internal categorisation I","Categorisation by connectivity","Technology VOD","Technology VOD: Auxiliar","Fibre / Microwave","Type of Structure","Tower Height (m)","Floor space","Floor space (availability)","Status","Infrastructure ready (existing)/ to be ready (new)","Infrastructure to be dismantled by","Date when Vodafone active equipment is removed","Infrastructure to be shared by","Core site type","Transmission Hub Site (YES/NO)","Transmission Hub Site (inc. with/without Shelters)","Transmission Site (inc. with/without Shelters)","Room configuration","An Indoor + Outdoor Site","Climate Control (YES/NO)","Air Conditioning (YES/NO)","Free-Air Cooling (YES/NO)","Power Supply ","Active/Passive DAS","Strategic Site (YES/NO)","Critical Site (YES/NO)","	If a Non-Critical and non-Transmission Hub Site, Category 1 or Category 2","Is the Site a WIP site","Indicative completion date?","Capital expenditure incurred to the MSA Effective Date","Indicative capital expenditure required to complete the site build","Vertical Infrastructure ","Site Type","Technology_VOD","Fibre/Microwave","Power Supply","# Antennas","# Minilinks","	Details of Operator Equipment installed at the Site","Site Configuration","	NOS shared site (Yes/No)","	MEO shared site (Yes/No)","Details of any Active Sharing involving Operator applicable in respect of the Site (and, where applicable, the type of Active Sharing);","	Details of the access arrangements applicable at each Site (including any access restrictions and applicable public access requirements).","Sites under Hawkins arrangements (Yes/No)","Hawkins Sites (Yes/No)","NOS Crossed Site (Yes/No)","BTS Sites (Yes/No)","Energy Provider ","POD ID","Annual Energy Consumption (kWh)","Annual Energy cost (€)","Total Energy Cost (Energy provider + Landlord)","# Lease Contracts","Countepart","Total Annual Lease (€)","Current annual lease fees (€)","Current energy lease fees (€)","Current annual other fees (€)","Annual Lease fees + Annual Other fees","Sub-Lease","(Latest) residual duration until expiring date","Maturity Cluster - Expiring date","(Latest) residual duration until expiring date after renewal","Maturity Cluster - Expiring date after renewal","Type of lease contract","Reason for no lease fee","Comments to Lease Contracts","NOS (1/0)","Total Revenues NOS (€)","Annual Hosting fee NOS (€)","Annual Energy fee NOS (€)","Annual Maintenance fee NOS (€)","Annual Other Services fee NOS (€)","Tenancy duration until expiring date NOS","Maturity Clusters NOS","MEO (1/0)","Total Revenues MEO (€)","Annual Hosting fee MEO (€)","Annual Energy fee MEO (€)","Annual Maintenance fee MEO (€)","Annual Other Services fee MEO (€)","Tenancy duration until expiring date MEO","Maturity Clusters MEO","OTMOs (1/0)","Total Revenues OTMOs (€)","Annual Hosting fee OTMOs (€)","Annual Energy fee OTMOs (€)","Annual Maintenance fee OTMOs (€)","Annual Other Services fee OTMOs (€)","Tenancy duration until expiring date OTMOs","Maturity Clusters OTMOs","Total # of 3rd Party Tenants","Name of 3rd Party Tenants","Type of Sharing","Total Hosting Fee & Services from 3rd Party Tenants (€)","Annual Hosting fee from 3rd Party Tenants (€)","Annual Energy fee from 3rd Party Tenants (€)","Annual Maintenance fee from 3rd Party Tenants (€)","Other Services fee from 3rd Party Tenants (€)","Weighted Average tenancy duration until expiring date","Total # of Tenants","Macro Cluster Tenancy","Comments to Tenant Agreements","Macro Cluster Type of contract","Easement (Servitù di passaggio)","Turistic Sites","Sites_As_Metered_Estimated","Strategic_Site_Bucket","Critical_Site_Beyond_10","First_Active_Sharing_Deployment_Type ","First_Active_Sharing_Start_Date","First_Active_Sharing_End_Date ","Subsequent_Sharing_Arrangement","Legacy_Site_Agreement_Terminated(Yes/NO)","Decommissioned Sites(True/false)"]
columns_integer_convert = ['Nr. Inhabitants','Tower Height (m)', 'Floor space','Floor space (availability)','Annual Energy Consumption (kWh)','Annual Energy cost (€)','Total Energy Cost (Energy provider + Landlord)','Current annual lease fees (€)','Current energy lease fees (€)','Current annual other fees (€)','Annual Lease fees + Annual Other fees','Total Revenues NOS (€)','Annual Hosting fee NOS (€)','Annual Energy fee NOS (€)','Annual Maintenance fee NOS (€)','Annual Other Services fee NOS (€)','OTMOs (1/0)','Total Revenues OTMOs (€)','Annual Hosting fee OTMOs (€)','Annual Energy fee OTMOs (€)','Annual Maintenance fee OTMOs (€)','Annual Other Services fee OTMOs (€)','MEO (1/0)','Total Revenues MEO (€)','Annual Hosting fee MEO (€)','Annual Energy fee MEO (€)','Annual Maintenance fee MEO (€)','Annual Other Services fee MEO (€)','Total Hosting Fee & Services from 3rd Party Tenants (€)','Annual Hosting fee from 3rd Party Tenants (€)','Annual Energy fee from 3rd Party Tenants (€)','Annual Maintenance fee from 3rd Party Tenants (€)','Other Services fee from 3rd Party Tenants (€)','Total # of Tenants', 'First_Active_Sharing_Deployment_Type ']
towerdb = read_files(test, sheetname, skiprows, skipcolumns,columns_integer_convert)
towerdb = towerdb[list_columns]

# Lendo o ficheiro de output para cruzar com o ficheiro de input
path_df_output = '/content/TowerDB_Portugal_20210630.csv'
df_output = pd.read_csv(path_df_output, encoding='latin')
df_output = df_output[['Site Code', 'Date_Of_Equipment_Removal']]


# Merge files to get Date of equipament removal columns from output file
towerdb = towerdb.merge(df_output, on='Site Code')

#Change datetime format
date_format = '%d/%m/%Y'
columns_date_convert = ['Infrastructure ready (existing)/ to be ready (new)','Date when Vodafone active equipment is removed','Infrastructure to be shared by', 'First_Active_Sharing_Start_Date',	'First_Active_Sharing_End_Date ','Subsequent_Sharing_Arrangement'] #, 'Date_Of_Equipment_Removal']
towerdb = change_date_format(towerdb, columns_date_convert, date_format)

# On the air sites which have blank values
status_columns = 'Status'
status = 'In Service'
on_air_columns = ['Site Code', 'Categorization by Transmission Sys','Categorization by Site Type',\
                  'Sites_As_Metered_Estimated', 'Infrastructure ready (existing)/ to be ready (new)', \
                  'Climate Control (YES/NO)', 'Strategic Site (YES/NO)', 'Critical Site (YES/NO)',\
                  'Is the Site a WIP site', 'Power Supply ', 'Strategic_Site_Bucket', \
                  'Critical_Site_Beyond_10','Legacy_Site_Agreement_Terminated(Yes/NO)', \
                  'Decommissioned Sites(True/false)']

path_on_air = '/content/on_air_sites_blank_values.xlsx'
# how can I highlight the blank cell before convert to xlsx
on_air_sites_check(towerdb, on_air_columns, status_columns, status, path_on_air)


#towerdb.to_csv('/content/towerdb_pt_output.csv',index=False)
towerdb.head(2)



,Site Code,Site Name,EVO Site ID,Macro Region,Region,Province,Municipality,Nr. Inhabitants,Address,Altitude,Latitude,Longitude,Ground Register,Categorization by inhabitants,Categorization by Transmission Sys,Categorization by Transmission Sys (sub-cluster),Categorization by Site Type,Other internal categorisation I,Categorisation by connectivity,Technology VOD,Technology VOD: Auxiliar,Fibre / Microwave,Type of Structure,Tower Height (m),Floor space,Floor space (availability),Status,Infrastructure ready (existing)/ to be ready (new),Infrastructure to be dismantled by,Date when Vodafone active equipment is removed,Infrastructure to be shared by,Core site type,Transmission Hub Site (YES/NO),Transmission Hub Site (inc. with/without Shelters),Transmission Site (inc. with/without Shelters),Room configuration,An Indoor + Outdoor Site,Climate Control (YES/NO),Air Conditioning (YES/NO),Free-Air Cooling (YES/NO),...,Total Revenues MEO (€),Annual Hosting fee MEO (€),Annual Energy fee MEO (€),Annual Maintenance fee MEO (€),Annual Other Services fee MEO (€),Tenancy duration until expiring date MEO,Maturity Clusters MEO,OTMOs (1/0),Total Revenues OTMOs (€),Annual Hosting fee OTMOs (€),Annual Energy fee OTMOs (€),Annual Maintenance fee OTMOs (€),Annual Other Services fee OTMOs (€),Tenancy duration until expiring date OTMOs,Maturity Clusters OTMOs,Total # of 3rd Party Tenants,Name of 3rd Party Tenants,Type of Sharing,Total Hosting Fee & Services from 3rd Party Tenants (€),Annual Hosting fee from 3rd Party Tenants (€),Annual Energy fee from 3rd Party Tenants (€),Annual Maintenance fee from 3rd Party Tenants (€),Other Services fee from 3rd Party Tenants (€),Weighted Average tenancy duration until expiring date,Total # of Tenants,Macro Cluster Tenancy,Comments to Tenant Agreements,Macro Cluster Type of contract,Easement (Servitù di passaggio),Turistic Sites,Sites_As_Metered_Estimated,Strategic_Site_Bucket,Critical_Site_Beyond_10,First_Active_Sharing_Deployment_Type,First_Active_Sharing_Start_Date,First_Active_Sharing_End_Date,Subsequent_Sharing_Arrangement,Legacy_Site_Agreement_Terminated(Yes/NO),Decommissioned Sites(True/false),Date_Of_Equipment_Removal
0,1,BRAGA,PT11000001,CONTINENT,NORTE,Braga,Braga,182679,"Travessa Escola, 4705-474 Esporões",518,41.516600,-8.395440,NaN,Urban,Macro,Standard,GBT,NaN,NaN,2G; 3G; 4G,NaN,Fibre,Lattice,51,240,223,In Service,01/01/1992,NaT,NaN,NaN,Non Core,Yes,With shelters,Non Transmission Site,Indoor,NO,Yes; Indoor Air Conditioning,Yes,No,...,,,,,,NaN,NaN,,,,,,,NaN,NaN,0,NaN,NaN,,,,,,NaN,1,VF Only,NaN,Leased Area - VF Only,NaN,NaN,Estimated Model,Non Strategic,Within 10%,,NaN,NaN,NaN,NaN,NaN,NaN
1,2,RECEZINHOS,PT13000002,CONTINENT,NORTE,Porto,Penafiel,69772,"Cruzes -Croca - Penafiel, 4560-871 Siforno",411,41.218686,-8.229177,NaN,Urban,Macro,Standard,GBT,NaN,NaN,2G; 3G; 4G,NaN,Fibre,Monopole,26,80,71,In Service,09/02/1992,NaT,NaN,NaN,Non Core,No,Non Transmission Hub Site,Non Transmission Site,Indoor,NO,No,No,No,...,,,,,,NaN,NaN,,,,,,,NaN,NaN,0,NaN,NaN,,,,,,NaN,1,VF Only,NaN,Leased Area - VF Only,NaN,NaN,Estimated Model,Non Strategic,Non Critical,,NaN,NaN,NaN,No,No,NaN


In [97]:
def on_air_sites_check(df, columns, status_column, status):
    
    # dataframe with a conditional setted
    status_df = df[df[status_column]==status]

    # Empty Dataframe to receive blank lines
    new_df = pd.DataFrame()

    aux = []
    for i in columns:
        df = status_df[status_df[i].isnull()]
        if not df.empty:
            aux.append(df)

    new_df = pd.concat(aux)
    new_df.drop_duplicates(inplace=True)
    return new_df

status_columns = 'Status'
status = 'In Service'
on_air_columns = ['Site Code', 'Categorization by Transmission Sys','Categorization by Site Type',\
                  'Sites_As_Metered_Estimated', 'Infrastructure ready (existing)/ to be ready (new)', \
                  'Climate Control (YES/NO)', 'Strategic Site (YES/NO)', 'Critical Site (YES/NO)',\
                  'Is the Site a WIP site', 'Power Supply ', 'Strategic_Site_Bucket', \
                  'Critical_Site_Beyond_10','Legacy_Site_Agreement_Terminated(Yes/NO)', \
                  'Decommissioned Sites(True/false)']
teste = on_air_sites_check(towerdb, on_air_columns, status_columns, status)
teste.head()




,Site Code,Site Name,EVO Site ID,Macro Region,Region,Province,Municipality,Nr. Inhabitants,Address,Altitude,Latitude,Longitude,Ground Register,Categorization by inhabitants,Categorization by Transmission Sys,Categorization by Transmission Sys (sub-cluster),Categorization by Site Type,Other internal categorisation I,Categorisation by connectivity,Technology VOD,Technology VOD: Auxiliar,Fibre / Microwave,Type of Structure,Tower Height (m),Floor space,Floor space (availability),Status,Infrastructure ready (existing)/ to be ready (new),Infrastructure to be dismantled by,Date when Vodafone active equipment is removed,Infrastructure to be shared by,Core site type,Transmission Hub Site (YES/NO),Transmission Hub Site (inc. with/without Shelters),Transmission Site (inc. with/without Shelters),Room configuration,An Indoor + Outdoor Site,Climate Control (YES/NO),Air Conditioning (YES/NO),Free-Air Cooling (YES/NO),...,Total Revenues MEO (€),Annual Hosting fee MEO (€),Annual Energy fee MEO (€),Annual Maintenance fee MEO (€),Annual Other Services fee MEO (€),Tenancy duration until expiring date MEO,Maturity Clusters MEO,OTMOs (1/0),Total Revenues OTMOs (€),Annual Hosting fee OTMOs (€),Annual Energy fee OTMOs (€),Annual Maintenance fee OTMOs (€),Annual Other Services fee OTMOs (€),Tenancy duration until expiring date OTMOs,Maturity Clusters OTMOs,Total # of 3rd Party Tenants,Name of 3rd Party Tenants,Type of Sharing,Total Hosting Fee & Services from 3rd Party Tenants (€),Annual Hosting fee from 3rd Party Tenants (€),Annual Energy fee from 3rd Party Tenants (€),Annual Maintenance fee from 3rd Party Tenants (€),Other Services fee from 3rd Party Tenants (€),Weighted Average tenancy duration until expiring date,Total # of Tenants,Macro Cluster Tenancy,Comments to Tenant Agreements,Macro Cluster Type of contract,Easement (Servitù di passaggio),Turistic Sites,Sites_As_Metered_Estimated,Strategic_Site_Bucket,Critical_Site_Beyond_10,First_Active_Sharing_Deployment_Type,First_Active_Sharing_Start_Date,First_Active_Sharing_End_Date,Subsequent_Sharing_Arrangement,Legacy_Site_Agreement_Terminated(Yes/NO),Decommissioned Sites(True/false),Date_Of_Equipment_Removal
0,1,BRAGA,PT11000001,CONTINENT,NORTE,Braga,Braga,182679,"Travessa Escola, 4705-474 Esporões",518,41.51660,-8.395440,NaN,Urban,Macro,Standard,GBT,NaN,NaN,2G; 3G; 4G,NaN,Fibre,Lattice,51,240,223,In Service,01/01/1992,NaT,NaN,NaN,Non Core,Yes,With shelters,Non Transmission Site,Indoor,NO,Yes; Indoor Air Conditioning,Yes,No,...,,,,,,NaN,NaN,,,,,,,NaN,NaN,0,NaN,NaN,,,,,,NaN,1,VF Only,NaN,Leased Area - VF Only,NaN,NaN,Estimated Model,Non Strategic,Within 10%,,NaN,NaN,NaN,NaN,NaN,NaN
3567,67575,CARVOEIRO PAMPILHOSA SUBST,PT21067575,CONTINENT,CENTRO-NORTE,Coimbra,Pampilhosa da Serra,4037,Rua Alberto Magno Oliveira (MUSEU ETNOGRAFICO)...,554,40.04329,-8.004722,NaN,Suburban,Macro,Standard,GBT,NaN,NaN,2G; 4G,NaN,Microwave,Lattice,30,,,In Service,03/09/2020,NaT,NaN,NaN,Non Core,No,Non Transmission Hub Site,Non Transmission Site,Outdoor,NO,No,No,No,...,,,,,,NaN,NaN,,,,,,,NaN,NaN,0,NaN,NaN,,,,,,NaN,1,VF Only,NaN,Leased Area - VF Only,NaN,NaN,Estimated Model,Non Strategic,Non Critical,,NaN,NaN,NaN,NaN,No,NaN


In [95]:

new_df = pd.DataFrame()
aux = []
for i in on_air_columns:
    df = on_air_df[on_air_df[i].isnull()]
    if not df.empty:
        print(df,i)
        aux.append(df)

new_df = pd.concat(aux)
new_df.drop_duplicates(inplace=True)
new_df.head()



      Site Code  ... Date_Of_Equipment_Removal
0             1  ...                       NaN
3567      67575  ...                       NaN

[2 rows x 135 columns] Legacy_Site_Agreement_Terminated(Yes/NO)
   Site Code  ... Date_Of_Equipment_Removal
0          1  ...                       NaN

[1 rows x 135 columns] Decommissioned Sites(True/false)


,Site Code,Site Name,EVO Site ID,Macro Region,Region,Province,Municipality,Nr. Inhabitants,Address,Altitude,Latitude,Longitude,Ground Register,Categorization by inhabitants,Categorization by Transmission Sys,Categorization by Transmission Sys (sub-cluster),Categorization by Site Type,Other internal categorisation I,Categorisation by connectivity,Technology VOD,Technology VOD: Auxiliar,Fibre / Microwave,Type of Structure,Tower Height (m),Floor space,Floor space (availability),Status,Infrastructure ready (existing)/ to be ready (new),Infrastructure to be dismantled by,Date when Vodafone active equipment is removed,Infrastructure to be shared by,Core site type,Transmission Hub Site (YES/NO),Transmission Hub Site (inc. with/without Shelters),Transmission Site (inc. with/without Shelters),Room configuration,An Indoor + Outdoor Site,Climate Control (YES/NO),Air Conditioning (YES/NO),Free-Air Cooling (YES/NO),...,Total Revenues MEO (€),Annual Hosting fee MEO (€),Annual Energy fee MEO (€),Annual Maintenance fee MEO (€),Annual Other Services fee MEO (€),Tenancy duration until expiring date MEO,Maturity Clusters MEO,OTMOs (1/0),Total Revenues OTMOs (€),Annual Hosting fee OTMOs (€),Annual Energy fee OTMOs (€),Annual Maintenance fee OTMOs (€),Annual Other Services fee OTMOs (€),Tenancy duration until expiring date OTMOs,Maturity Clusters OTMOs,Total # of 3rd Party Tenants,Name of 3rd Party Tenants,Type of Sharing,Total Hosting Fee & Services from 3rd Party Tenants (€),Annual Hosting fee from 3rd Party Tenants (€),Annual Energy fee from 3rd Party Tenants (€),Annual Maintenance fee from 3rd Party Tenants (€),Other Services fee from 3rd Party Tenants (€),Weighted Average tenancy duration until expiring date,Total # of Tenants,Macro Cluster Tenancy,Comments to Tenant Agreements,Macro Cluster Type of contract,Easement (Servitù di passaggio),Turistic Sites,Sites_As_Metered_Estimated,Strategic_Site_Bucket,Critical_Site_Beyond_10,First_Active_Sharing_Deployment_Type,First_Active_Sharing_Start_Date,First_Active_Sharing_End_Date,Subsequent_Sharing_Arrangement,Legacy_Site_Agreement_Terminated(Yes/NO),Decommissioned Sites(True/false),Date_Of_Equipment_Removal
0,1,BRAGA,PT11000001,CONTINENT,NORTE,Braga,Braga,182679,"Travessa Escola, 4705-474 Esporões",518,41.51660,-8.395440,NaN,Urban,Macro,Standard,GBT,NaN,NaN,2G; 3G; 4G,NaN,Fibre,Lattice,51,240,223,In Service,01/01/1992,NaT,NaN,NaN,Non Core,Yes,With shelters,Non Transmission Site,Indoor,NO,Yes; Indoor Air Conditioning,Yes,No,...,,,,,,NaN,NaN,,,,,,,NaN,NaN,0,NaN,NaN,,,,,,NaN,1,VF Only,NaN,Leased Area - VF Only,NaN,NaN,Estimated Model,Non Strategic,Within 10%,,NaN,NaN,NaN,NaN,NaN,NaN
3567,67575,CARVOEIRO PAMPILHOSA SUBST,PT21067575,CONTINENT,CENTRO-NORTE,Coimbra,Pampilhosa da Serra,4037,Rua Alberto Magno Oliveira (MUSEU ETNOGRAFICO)...,554,40.04329,-8.004722,NaN,Suburban,Macro,Standard,GBT,NaN,NaN,2G; 4G,NaN,Microwave,Lattice,30,,,In Service,03/09/2020,NaT,NaN,NaN,Non Core,No,Non Transmission Hub Site,Non Transmission Site,Outdoor,NO,No,No,No,...,,,,,,NaN,NaN,,,,,,,NaN,NaN,0,NaN,NaN,,,,,,NaN,1,VF Only,NaN,Leased Area - VF Only,NaN,NaN,Estimated Model,Non Strategic,Non Critical,,NaN,NaN,NaN,NaN,No,NaN


In [33]:
df = pd.DataFrame({'A' : [4,5,13,18], 'B' : [10,'','',40], 'C' : ['',50,25,''], 'D' : [-30,-50,10,16], 'E' : [-40,-50,7,12]})
df.isnull().sum()


A    0
B    0
C    0
D    0
E    0
dtype: int64

In [21]:
#Input Data
#towerdb.groupby('Is the Site a WIP site')['Site Code'].count()
wip_site = towerdb[['Site Code','Is the Site a WIP site']]
wip_site = wip_site[wip_site['Is the Site a WIP site'] == 'Yes']

bts_site = towerdb[['Site Code', 'BTS Sites (Yes/No)']]
bts_site = bts_site[bts_site['BTS Sites (Yes/No)'] == 'No']

legacy_site = towerdb[['Site Code', 'Legacy_Site_Agreement_Terminated(Yes/NO)']]
legacy_site = legacy_site[legacy_site['Legacy_Site_Agreement_Terminated(Yes/NO)'] == 'No']

#wip_site.count()
legacy_site.count()


Site Code                                   3570
Legacy_Site_Agreement_Terminated(Yes/NO)    3570
dtype: int64

In [58]:
#Replace NaN values from Annual Lease fees + Annual Other fees column
column = ['First_Active_Sharing_Start_Date']
conditional = np.nan
new_value = ' '

towerdb['First_Active_Sharing_Start_Date']


0              NaN
1              NaN
2              NaN
3              NaN
4              NaN
           ...    
3565           NaN
3566           NaN
3567           NaN
3568           NaN
3569    31/03/2015
Name: First_Active_Sharing_Start_Date, Length: 3570, dtype: object

In [5]:
df[columns_integer_convert] = df[columns_integer_convert].fillna(0)
df[columns_integer_convert] = df[columns_integer_convert].astype('int64')
df.head(2)

,Site Code,Site Name,EVO Site ID,Macro Region,Region,Province,Municipality,Nr. Inhabitants,Address,Altitude,Latitude,Longitude,Ground Register,Categorization by inhabitants,Categorization by Transmission Sys,Categorization by Transmission Sys (sub-cluster),Categorization by Site Type,Other internal categorisation I,Categorisation by connectivity,Technology VOD,Technology VOD: Auxiliar,Fibre / Microwave,Type of Structure,Tower Height (m),Floor space,Floor space (availability),Status,Infrastructure ready (existing)/ to be ready (new),Infrastructure to be dismantled by,Date when Vodafone active equipment is removed,Infrastructure to be shared by,Core site type,Transmission Hub Site (YES/NO),Transmission Hub Site (inc. with/without Shelters),Transmission Site (inc. with/without Shelters),Room configuration,An Indoor + Outdoor Site,Climate Control (YES/NO),Air Conditioning (YES/NO),Free-Air Cooling (YES/NO),...,Annual Hosting fee OTMOs (€),Annual Energy fee OTMOs (€),Annual Maintenance fee OTMOs (€),Annual Other Services fee OTMOs (€),Tenancy duration until expiring date OTMOs,Maturity Clusters OTMOs,Total # of 3rd Party Tenants,Name of 3rd Party Tenants,Type of Sharing,Total Hosting Fee & Services from 3rd Party Tenants (€),Annual Hosting fee from 3rd Party Tenants (€),Annual Energy fee from 3rd Party Tenants (€),Annual Maintenance fee from 3rd Party Tenants (€),Other Services fee from 3rd Party Tenants (€),Weighted Average tenancy duration until expiring date,Total # of Tenants,Macro Cluster Tenancy,Comments to Tenant Agreements,Macro Cluster Type of contract,Easement (Servitù di passaggio),Turistic Sites,Sites_As_Metered_Estimated,Strategic_Site_Bucket,Critical_Site_Beyond_10,First_Active_Sharing_Deployment_Type,First_Active_Sharing_Start_Date,First_Active_Sharing_End_Date,Subsequent_Sharing_Arrangement,Legacy_Site_Agreement_Terminated(Yes/NO),Decommissioned Sites(True/false),Renewed/Expired Lease,CO (Yes/No),TIMS Site Code,Climate Control ll,Number of landlords per site,RFAI Date,BTS Site Acceptance Date,Billing Trigger Date,Site Type.1,Billing Trigger Stop Date from May21 onwards
0,1,BRAGA,PT11000001,CONTINENT,NORTE,Braga,Braga,182679,"Travessa Escola, 4705-474 Esporões",518,41.516600,-8.395440,NaN,Urban,Macro,Standard,GBT,NaN,NaN,2G; 3G; 4G,NaN,Fibre,Lattice,51,240,223,In Service,1992-01-01,NaT,NaT,NaT,Non Core,Yes,With shelters,Non Transmission Site,Indoor,NO,Yes; Indoor Air Conditioning,Yes,No,...,0,0,0,0,NaN,NaN,0,NaN,NaN,0,0,0,0,0,NaN,1,VF Only,NaN,Leased Area - VF Only,NaN,NaN,Estimated Model,Non Strategic,Within 10%,NaN,NaN,NaN,NaN,No,No,Renewed lease,No,PT-TIMS-54223,"Yes, Indoor Air Conditioning",1,NaT,NaT,1992-01-01,LEGACY,NaT
1,2,RECEZINHOS,PT13000002,CONTINENT,NORTE,Porto,Penafiel,69772,"Cruzes -Croca - Penafiel, 4560-871 Siforno",411,41.218686,-8.229177,NaN,Urban,Macro,Standard,GBT,NaN,NaN,2G; 3G; 4G,NaN,Fibre,Monopole,26,80,71,In Service,1992-02-09,NaT,NaT,NaT,Non Core,No,Non Transmission Hub Site,Non Transmission Site,Indoor,NO,No,No,No,...,0,0,0,0,NaN,NaN,0,NaN,NaN,0,0,0,0,0,NaN,1,VF Only,NaN,Leased Area - VF Only,NaN,NaN,Estimated Model,Non Strategic,Non Critical,NaN,NaN,NaN,NaN,No,No,Renewed lease,No,PT-TIMS-54224,No,1,NaT,NaT,1992-02-09,LEGACY,NaT


In [6]:
path='/content/TowerDB_Ireland_20210430 v5_sem_password.xlsx'
sheetname = 'TowerDB_Input_Ireland_20210430'
skiprows = 0
skipcolumns = 0
towerdb_ie = read_files(path,sheetname, skiprows, skipcolumns, columns_integer_convert, columns_date_convert)



KeyboardInterrupt: ignored

In [ ]:
import numpy as np
import math
cont = 0
for i in towerdb_ie.iloc[:,0]:
    if str(i) == 'nan':
        break
    else:
        cont +=1
    #print(i)
print(cont)

In [ ]:
import pandas as pd
towerdb_pt = pd.read_excel('/content/Vantage Towers_PT TowerDB Apr21.xlsx', sheet_name='Final Delivery', skiprows=6)
#lc_de = pd.read_excel('/content/LC_teste.xlsx')
#ta_pt = pd.read_excel('/content/tenant.xlsx')

In [ ]:
towerdb_pt = towerdb_pt.iloc[:,1:]
#towerdb_pt.columns = [i.lower() for i in towerdb_pt.columns]
#towerdb_pt.head(2)
towerdb_pt.head(2)

In [ ]:
towerDB_de_output = ["Code","Site Name","Macro Region","Region","Province","Municipality","Inhabitants","Address",\
                     "Latitude","Longitude","Categorization by Transmission Sys",\
                     "Categorization by Transmission Sys (sub-cluster)","Categorization by Site Type",\
                     "Categorization by Indoor / Outdoor Site (Antennas)","Additional Site Type information",\
                     "Categorization by Inhabitants","Rural/ Suburban/ Urban","Categorization by CONNECTIVITY",\
                     "Technology VOD","POD ID","Energy Consumption current FY","Actual Energy Cost current FY",\
                     "Infrastructure ready (existing)/ to be ready (new)","Infrastructure to be shared by","Counterpart",\
                     "# of Lease Contracts","Current annual lease fees","Current energy lease fees",\
                     "Current annual other fees","Total Annual Lease","(Average) residual duration","Maturity Cluster",\
                     "ExCo rep. Avg Annual Lease costs","Total Energy Cost currrent FY (Energy provider + LL)","VOD (y/n)",\
                     "Deutsche Telekom","Annual Fee per Tenant MNO1","Annual Energy Fee MNO1","Annual Maintenance Fee MNO1",\
                     "Other Services Fee MNO1","Total Hosting Fee & Services MNO1","Residual duration MNO1",\
                     "Maturity Clusters MNO1","Telefonica","Annual Fee per Tenant MNO2","Annual Energy Fee MNO2",\
                     "Annual Maintenance Fee MNO2","Other Services Fee MNO2","Total Hosting Fee & Services MNO2",\
                     "Residual duration MNO2","Maturity Clusters MNO2","E-Plus","Annual Fee per Tenant MNO3",\
                     "Annual Energy Fee MNO3","Annual Maintenance Fee MNO3","Other Services Fee MNO3",\
                     "Total Hosting Fee & Services","Residual duration MNO3","Maturity Clusters","# of OTMOs",\
                     "Annual Fee from OTMOs","Annual Energy Fee from OTMOs","Annual Maintenance Fee OTMOs",\
                     "Other Services Fee OTMOs","Total Hosting Fee & Services OTMOs","Average residual duration OTMOs",\
                     "Maturity Clusters OTMOs","Total # of 3rd Party Tenants","Annual Fee from 3rd Party Tenants",\
                     "Annual Energy Fee from 3rd Party Tenants","Annual Maintenance Fee from 3rd Party Tenants",\
                     "Other Services Fee from 3rd Party Tenants","Total Hosting Fee & Services from 3rd Party Tenants",\
                     "Weighted Average residual duration","Macro Cluster Tenancy",\
                     "Macro Cluster Lease / Freeholds & Surface Right","Macro Cluster 1",\
                     "Sites w/ at least a DDS (Lease Contract Type)","# of Tenants","Categorization by Tenant combination",\
                     "Categorization by Type of Passive contracts","Categorization by Land/Surface ownership",\
                     "In/Out","No of 3rd Party Tenants (PoP count)","Top cities classification","Type of Structure",\
                     "Categorization by shared usage (contractual)","Categorization by Shared Usage (PoP count)",\
                     "Security Class","Categorization by Sites with Single/Multiple Tower(s)",\
                     "Categorization by Special Site/PLC","Categorization by Chimney-location",\
                     "Categorization by Antenna Constraint","Categorization by 5G Roll-out Site",\
                     "Categorization by Authority Site","Categorization by Transport-Sammler",\
                     "Categorization by Strategic Site","Categorization by Dual Use Site",\
                     "Categorization by Sublease Right","GBT Reserve Space","State","Postal code","Tower Height",\
                     "Floor Space","Ground Register #1","Ground Register #2","Ground Register #3",\
                     "Categorization by Critical Site","Site Cluster","infrastructure to be dismantled by",\
                     "decommissioned site","Strategic Site Bucket","Bts_sites","CriticalSite_Beyond_10",\
                     "Sites As Metered Estimated","Active Sharing Arrangement","Subsequent_Active_Sharing_Arrangement",\
                     "First_Active_Sharing_Deployment_Type","First_Active_Sharing_Arrangement_Start_Date",\
                     "First_Active_Sharing_Arrangement_End_Date","Indoor_Site_Any_Climate_Control",\
                     "Outdoor_Site_With_Dc_Power","Rfai_Date","Cluster for non-enterprise DAS Sites",\
                     "Site Status (TIMS)","Date_Of_Equipment_Removal"]

ta_de_output = ["Code","Tenant Agreement","Tenant","Tenant Cluster","MNO Cluster","Tenant (ID)",\
                "Starting date","Expiring date",'" Annual Fee per Tenant "',"Overlapping days in FY19",\
                '" Current Annual Fee per Tenant "',"Annual Energy Fee from 3rd Party Tenants",'" OTP Amount "',\
                '" Renewal option "','" Residual duration "',"Maturity Cluster","Contract Scope","Site Scope",\
                "Contract Migration Scope 14.05.2020","Comment"]

lc_de_output = ["Lease Contract ID","Code","Counterpart","Counterpart Cluster","Counterpart (ID)","Starting date",\
                "Expiring date",'" Annual lease fees "',"Overlapping days in FY20",'" Current annual lease fees "',\
                '" OTP Amount "',"Renewal option","(Average) residual duration","Maturity Cluster","Site Scope",\
                "Contract scope","MNO Cluster","reason for scope","Adjustment","+/-","Contract Migration Scope 14.05.2020"]

lower_towerdb_columns = [i.lower() for i in towerDB_de_output] 
towerdb_de.columns = [i.lower() for i in tenant.columns]
towerdb_de = towerdb_de.reindex(columns=lower_towerdb_columns)


In [ ]:
towerdb_de.head(10) 

In [ ]:
import numpy as np
#create an empty DF with billing columns required
towerdb_de_final = pd.DataFrame(columns=[i.lower() for i in towerDB_de_output])
for i in [i.lower() for i in towerDB_de_output]:
    #if towerdb_de_final[i].equals(tenant[i]) == True:
    towerdb_de_final[i] = tenant[i].values
towerdb_de_final.head(2)

In [ ]:
towerdb_de_final['Site Name'].equals(tenant[i])

In [ ]:
"""
tower_total_count, tower_count, tower_missing_columns = check_columns(df, towerDB_de_output)
print(log_output.format(sheetname='TowerDB',total_received = tower_total_count, \
                        number_missing_columns = tower_count, \
                        columns = '\n'.join('- '+ i for i in tower_missing_columns)),\
      file=open('/content/tower_log.txt','a')) 
"""
# read pt files
towerdb_pt = pd.read_excel('/content/towerdb.xlsx')
tenant_pt = pd.read_excel('/content/tenant.xlsx')     

towerdb_pt_output = ["Site Code","Site Name","EVO Site ID","Macro Region","Region","Province","Municipality","Nr. Inhabitants","Address","Altitude","Latitude","Longitude","Ground Register","Categorization by inhabitants","Categorization by Transmission Sys","Categorization by Transmission Sys (sub-cluster)","Categorization by Site Type","Other internal categorisation I","Categorisation by connectivity","Technology VOD","Technology VOD: Auxiliar","Fibre / Microwave","Type of Structure","Tower Height (m)","Floor space","Floor space (availability)","Status","Infrastructure ready (existing)/ to be ready (new)","Infrastructure to be dismantled by","Date when Vodafone active equipment is removed","Infrastructure to be shared by","Core site type","Transmission Hub Site (YES/NO)","Transmission Hub Site (inc. with/without Shelters)","Transmission Site (inc. with/without Shelters)","Room configuration","An Indoor + Outdoor Site","Climate Control (YES/NO)","Air Conditioning (YES/NO)","Free-Air Cooling (YES/NO)","Power Supply ","Active/Passive DAS","Strategic Site (YES/NO)","Critical Site (YES/NO)","	If a Non-Critical and non-Transmission Hub Site, Category 1 or Category 2","Is the Site a WIP site","Indicative completion date?","Capital expenditure incurred to the MSA Effective Date","Indicative capital expenditure required to complete the site build","Vertical Infrastructure ","Site Type","Technology_VOD","Fibre/Microwave","Power Supply","# Antennas","# Minilinks","	Details of Operator Equipment installed at the Site","Site Configuration","	NOS shared site (Yes/No)","	MEO shared site (Yes/No)","Details of any Active Sharing involving Operator applicable in respect of the Site (and, where applicable, the type of Active Sharing);","	Details of the access arrangements applicable at each Site (including any access restrictions and applicable public access requirements).","Sites under Hawkins arrangements (Yes/No)","Hawkins Sites (Yes/No)","NOS Crossed Site (Yes/No)","BTS Sites (Yes/No)","Energy Provider ","POD ID","Annual Energy Consumption (kWh)","Annual Energy cost (€)","Total Energy Cost (Energy provider + Landlord)","# Lease Contracts","Countepart","Total Annual Lease (€)","Current annual lease fees (€)","Current energy lease fees (€)","Current annual other fees (€)","Annual Lease fees + Annual Other fees","Sub-Lease","(Latest) residual duration until expiring date","Maturity Cluster - Expiring date","(Latest) residual duration until expiring date after renewal","Maturity Cluster - Expiring date after renewal","Type of lease contract","Reason for no lease fee","Comments to Lease Contracts","NOS (1/0)","Total Revenues NOS (€)","Annual Hosting fee NOS (€)","Annual Energy fee NOS (€)","Annual Maintenance fee NOS (€)","Annual Other Services fee NOS (€)","Tenancy duration until expiring date NOS","Maturity Clusters NOS","MEO (1/0)","Total Revenues MEO (€)","Annual Hosting fee MEO (€)","Annual Energy fee MEO (€)","Annual Maintenance fee MEO (€)","Annual Other Services fee MEO (€)","Tenancy duration until expiring date MEO","Maturity Clusters MEO","OTMOs (1/0)","Total Revenues OTMOs (€)","Annual Hosting fee OTMOs (€)","Annual Energy fee OTMOs (€)","Annual Maintenance fee OTMOs (€)","Annual Other Services fee OTMOs (€)","Tenancy duration until expiring date OTMOs","Maturity Clusters OTMOs","Total # of 3rd Party Tenants","Name of 3rd Party Tenants","Type of Sharing","Total Hosting Fee & Services from 3rd Party Tenants (€)","Annual Hosting fee from 3rd Party Tenants (€)","Annual Energy fee from 3rd Party Tenants (€)","Annual Maintenance fee from 3rd Party Tenants (€)","Other Services fee from 3rd Party Tenants (€)","Weighted Average tenancy duration until expiring date","Total # of Tenants","Macro Cluster Tenancy","Comments to Tenant Agreements","Macro Cluster Type of contract","Easement (Servitù di passaggio)","Turistic Sites","Sites_As_Metered_Estimated","Strategic_Site_Bucket","Critical_Site_Beyond_10","First_Active_Sharing_Deployment_Type ","First_Active_Sharing_Start_Date","First_Active_Sharing_End_Date ","Subsequent_Sharing_Arrangement","Legacy_Site_Agreement_Terminated(Yes/NO)","Decommissioned Sites(True/false)","Date_Of_Equipment_Removal"]
tenant_pt_output = ["VF Site code","VF Site name","Other operator Site code","Other operator Site name","OWNER","TENANT","Tenant classification","Type of sharing","Annual Hosting Fee (in €)","M1","M2","M3","M4","M5","M6","M7","M8","M9","M10","M11","M12","Annual Energy Fee (€)","Annual Maintenance Fee (€)","Annual Other Services Fee (€)","Total Annual Fee (€)","Authorisation Date","Fee Starting Date","Fee Expiring Date","Renewal Option","Expiring date after renewal","Termination Date","Terms of payments","Payment type","Index","Indexation Driver","Percentage","VAT Subject","Percentage (VAT)","Inputs from filling responsible","Tenancy duration until expiring date (in years)","Sharing In Scope","CONTINENT"]
#Tenant file needs change the column names to the correct format, removing the Euro acronym

# checking towerdb columns
tower_tr, tower_nmc, tower_mc  = check_columns(towerdb_pt, towerdb_pt_output)
# saving towerdb log_file 
print(log_output.format(sheetname='TowerDB_PT',total_received = tower_tr, number_missing_columns = tower_nmc,\
                  columns = '\n'.join('- '+ i for i in tower_mc)),file=open('/content/towerdb_pt_log.txt','w'))

# checking tenant columns
tenant_tr, tenant_nmc, tenant_mc  = check_columns(tenant_pt, tenant_pt_output)
# saving tenant log_file 
print(log_output.format(sheetname='Tenant_PT',total_received = tenant_tr, number_missing_columns = tenant_nmc,\
                  columns = '\n'.join('- '+ i for i in tenant_mc)),file=open('/content/tenant_pt_log.txt','w'))

In [ ]:

ta_total_count, ta_count, ta_missing_columns = check_columns(ta, ta_de_output)
print(log_output.format(sheetname='TA_Input',total_received = ta_total_count, number_missing_columns = ta_count, \
                        columns = '\n'.join('- '+ i for i in ta_missing_columns)),\
      file=open('/content/ta_input_log.txt','a'))               
 

In [ ]:


lc_total_count, lc_count, lc_missing_columns = check_columns(lc, lc_de_output)
print(log_output.format(sheetname='LC_Input',total_received = lc_total_count, number_missing_columns = lc_count,\
                        columns = '\n'.join('- '+ i for i in lc_missing_columns)),\
      file=open('/content/lc_input_log.txt','a'))               
